In [3]:
!pip install feedparser

In [3]:
import feedparser
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from bson.objectid import ObjectId

load_dotenv('../.env');

In [4]:
mongo_host = os.environ.get("MONGO_HOST")
mongo_user = os.environ.get("MONGO_USER")
mongo_pass = os.environ.get("MONGO_PASS")
mongo_url  = f"mongodb+srv://{mongo_user}:{mongo_pass}@{mongo_host}/?retryWrites=true&w=majority"

def get_database(url, dbname, appname=None):
    try:
        # host and port default to 'localhost' and '27017'
        # appname will appear in logs
        client = MongoClient(url, appname=appname)
        # try connection to the database
        # The ping command is cheap and does not require auth.
        client.admin.command('ping')
        return client[dbname] 
    
    except ConnectionFailure:
        print("Error: MongoDB Server not available")


db = get_database(mongo_url, 'capstone', 'Capstone')


In [26]:
# https://blog.investmentpal.com/rss-feeds-finance-sites
feeds = {
    'wsj' : 'https://feeds.a.dj.com/rss/RSSMarketsMain.xml',
    'bloomberg' : 'https://feeds.bloomberg.com/markets/news.rss',
    'marketwatch' : 'https://feeds.content.dowjones.io/public/rss/mw_realtimeheadlines',
    'investopedia' : 'https://www.investopedia.com/feedbuilder/feed/getfeed?feedName=rss_articles',
    'cnbc' : 'https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=10000664',
    'huffpost' : 'https://www.huffpost.com/section/money/feed',
    'cbs' : 'https://www.cbsnews.com/latest/rss/moneywatch',
    'politico' : 'https://rss.politico.com/morningmoney.xml',
    'thehill' : 'https://thehill.com/policy/finance/feed',
    'time' : 'https://time.com/nextadvisor/feed',
    'seattletimes' : 'https://www.seattletimes.com/business/feed',
    'economist' : 'http://www.economist.com/feeds/print-sections/77/business.xml',
    'nasdaq' : 'https://www.nasdaq.com/feed/rssoutbound?category=Stocks',
    'cnn' : 'http://rss.cnn.com/rss/money_markets.rss',
    'investors_daily' : 'https://www.investors.com/feed',
    'fortune' : 'https://fortune.com/section/finance/feed',
    'market_beat' : 'https://www.marketbeat.com/feed',
    'kiplinger' : 'https://www.kiplinger.com/feed/all',
    'money' : 'https://money.com/money/feed',
    '247wallst' : 'https://247wallst.com/investing/feed',
    'nakedcapitalism' : 'https://www.nakedcapitalism.com/feed',
    'nar' : 'http://feeds.feedburner.com/nar/economists-outlook',
    'storey' : 'https://storeys.com/money/feed',
    'thinkadvisor' : 'https://feeds.feedblitz.com/ThinkAdvisor',
    'investmentnews' : 'https://www.investmentnews.com/feed',
    'dailyreckoning': 'https://dailyreckoning.com/feed',
    'wealthretirement' : 'https://wealthyretirement.com/feed',
}



In [ ]:
for name, feed in feeds.items():
    d = feedparser.parse(feed)

    for item in d['items']:
        db.headlines.insert_one({
            'channel': d['feed']['title'],
            'title': item['title'],
            'text': item['description'],
            'date': item['published']
        })
